In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

In [6]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(2):
            self.model.add(dense(256, weight_init))
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

In [7]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(2) :
        model.add(dense(256, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

In [9]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_xavier'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

In [10]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):                
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...


C:\Users\jihyu\anaconda3\lib\site-packages\keras\initializers\initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


Epoch: [ 0] [    0/  468] time: 0.6180, train_loss: 2.05240202, train_accuracy: 0.3438, test_Accuracy: 0.2872
Epoch: [ 0] [    1/  468] time: 0.8015, train_loss: 1.98501611, train_accuracy: 0.4922, test_Accuracy: 0.4121
Epoch: [ 0] [    2/  468] time: 0.9706, train_loss: 1.83527637, train_accuracy: 0.5547, test_Accuracy: 0.5359
Epoch: [ 0] [    3/  468] time: 1.1617, train_loss: 1.74565589, train_accuracy: 0.6406, test_Accuracy: 0.6133
Epoch: [ 0] [    4/  468] time: 1.3558, train_loss: 1.58071876, train_accuracy: 0.6641, test_Accuracy: 0.6603
Epoch: [ 0] [    5/  468] time: 1.5188, train_loss: 1.40244985, train_accuracy: 0.7422, test_Accuracy: 0.6820
Epoch: [ 0] [    6/  468] time: 1.6994, train_loss: 1.35001874, train_accuracy: 0.7344, test_Accuracy: 0.7187
Epoch: [ 0] [    7/  468] time: 1.8639, train_loss: 1.32910001, train_accuracy: 0.7188, test_Accuracy: 0.7461
Epoch: [ 0] [    8/  468] time: 2.0395, train_loss: 1.12807989, train_accuracy: 0.7812, test_Accuracy: 0.7374
Epoch: [ 0

Epoch: [ 0] [   75/  468] time: 13.7245, train_loss: 0.41927916, train_accuracy: 0.8984, test_Accuracy: 0.9152
Epoch: [ 0] [   76/  468] time: 13.9139, train_loss: 0.31550896, train_accuracy: 0.9219, test_Accuracy: 0.9172
Epoch: [ 0] [   77/  468] time: 14.1047, train_loss: 0.33166382, train_accuracy: 0.9141, test_Accuracy: 0.9205
Epoch: [ 0] [   78/  468] time: 14.2718, train_loss: 0.29143608, train_accuracy: 0.9219, test_Accuracy: 0.9216
Epoch: [ 0] [   79/  468] time: 14.4548, train_loss: 0.20948873, train_accuracy: 0.9297, test_Accuracy: 0.9210
Epoch: [ 0] [   80/  468] time: 14.6248, train_loss: 0.26249996, train_accuracy: 0.9375, test_Accuracy: 0.9178
Epoch: [ 0] [   81/  468] time: 14.7988, train_loss: 0.34951723, train_accuracy: 0.9062, test_Accuracy: 0.9167
Epoch: [ 0] [   82/  468] time: 14.9651, train_loss: 0.28087351, train_accuracy: 0.8984, test_Accuracy: 0.9131
Epoch: [ 0] [   83/  468] time: 15.1275, train_loss: 0.31606978, train_accuracy: 0.8906, test_Accuracy: 0.9133
E

Epoch: [ 0] [  150/  468] time: 27.7236, train_loss: 0.14342014, train_accuracy: 0.9531, test_Accuracy: 0.9353
Epoch: [ 0] [  151/  468] time: 27.9033, train_loss: 0.18492801, train_accuracy: 0.9531, test_Accuracy: 0.9343
Epoch: [ 0] [  152/  468] time: 28.0914, train_loss: 0.15159316, train_accuracy: 0.9609, test_Accuracy: 0.9347
Epoch: [ 0] [  153/  468] time: 28.2781, train_loss: 0.18839528, train_accuracy: 0.9609, test_Accuracy: 0.9371
Epoch: [ 0] [  154/  468] time: 28.4484, train_loss: 0.19234118, train_accuracy: 0.9297, test_Accuracy: 0.9385
Epoch: [ 0] [  155/  468] time: 28.6058, train_loss: 0.17610948, train_accuracy: 0.9375, test_Accuracy: 0.9380
Epoch: [ 0] [  156/  468] time: 28.7878, train_loss: 0.21143809, train_accuracy: 0.9375, test_Accuracy: 0.9367
Epoch: [ 0] [  157/  468] time: 28.9673, train_loss: 0.22921878, train_accuracy: 0.9531, test_Accuracy: 0.9388
Epoch: [ 0] [  158/  468] time: 29.1324, train_loss: 0.18901479, train_accuracy: 0.9453, test_Accuracy: 0.9376
E

Epoch: [ 0] [  225/  468] time: 41.1056, train_loss: 0.15031515, train_accuracy: 0.9531, test_Accuracy: 0.9386
Epoch: [ 0] [  226/  468] time: 41.2723, train_loss: 0.14803793, train_accuracy: 0.9531, test_Accuracy: 0.9381
Epoch: [ 0] [  227/  468] time: 41.4385, train_loss: 0.21084726, train_accuracy: 0.9531, test_Accuracy: 0.9418
Epoch: [ 0] [  228/  468] time: 41.6001, train_loss: 0.29347071, train_accuracy: 0.8984, test_Accuracy: 0.9447
Epoch: [ 0] [  229/  468] time: 41.7664, train_loss: 0.12513982, train_accuracy: 0.9688, test_Accuracy: 0.9475
Epoch: [ 0] [  230/  468] time: 41.9343, train_loss: 0.10204440, train_accuracy: 0.9688, test_Accuracy: 0.9492
Epoch: [ 0] [  231/  468] time: 42.1016, train_loss: 0.25364381, train_accuracy: 0.9062, test_Accuracy: 0.9482
Epoch: [ 0] [  232/  468] time: 42.2583, train_loss: 0.04939666, train_accuracy: 0.9922, test_Accuracy: 0.9461
Epoch: [ 0] [  233/  468] time: 42.4368, train_loss: 0.08029602, train_accuracy: 0.9766, test_Accuracy: 0.9442
E

Epoch: [ 0] [  300/  468] time: 55.7889, train_loss: 0.20139274, train_accuracy: 0.9609, test_Accuracy: 0.9545
Epoch: [ 0] [  301/  468] time: 55.9624, train_loss: 0.14036171, train_accuracy: 0.9531, test_Accuracy: 0.9527
Epoch: [ 0] [  302/  468] time: 56.1319, train_loss: 0.28395066, train_accuracy: 0.9297, test_Accuracy: 0.9498
Epoch: [ 0] [  303/  468] time: 56.3014, train_loss: 0.25680014, train_accuracy: 0.9297, test_Accuracy: 0.9480
Epoch: [ 0] [  304/  468] time: 56.5446, train_loss: 0.13812941, train_accuracy: 0.9453, test_Accuracy: 0.9476
Epoch: [ 0] [  305/  468] time: 56.7607, train_loss: 0.17609313, train_accuracy: 0.9609, test_Accuracy: 0.9494
Epoch: [ 0] [  306/  468] time: 56.9426, train_loss: 0.19694540, train_accuracy: 0.9453, test_Accuracy: 0.9514
Epoch: [ 0] [  307/  468] time: 57.1456, train_loss: 0.21256611, train_accuracy: 0.9453, test_Accuracy: 0.9537
Epoch: [ 0] [  308/  468] time: 57.4224, train_loss: 0.05638434, train_accuracy: 0.9844, test_Accuracy: 0.9540
E

Epoch: [ 0] [  374/  468] time: 70.4546, train_loss: 0.08337273, train_accuracy: 0.9844, test_Accuracy: 0.9533
Epoch: [ 0] [  375/  468] time: 70.6301, train_loss: 0.18931949, train_accuracy: 0.9531, test_Accuracy: 0.9525
Epoch: [ 0] [  376/  468] time: 70.8122, train_loss: 0.16639134, train_accuracy: 0.9453, test_Accuracy: 0.9521
Epoch: [ 0] [  377/  468] time: 71.0408, train_loss: 0.15723744, train_accuracy: 0.9609, test_Accuracy: 0.9542
Epoch: [ 0] [  378/  468] time: 71.2242, train_loss: 0.13894176, train_accuracy: 0.9688, test_Accuracy: 0.9553
Epoch: [ 0] [  379/  468] time: 71.4243, train_loss: 0.20484775, train_accuracy: 0.9531, test_Accuracy: 0.9563
Epoch: [ 0] [  380/  468] time: 71.6268, train_loss: 0.23652752, train_accuracy: 0.9141, test_Accuracy: 0.9590
Epoch: [ 0] [  381/  468] time: 71.8089, train_loss: 0.07743688, train_accuracy: 0.9766, test_Accuracy: 0.9590
Epoch: [ 0] [  382/  468] time: 72.0159, train_loss: 0.09777646, train_accuracy: 0.9688, test_Accuracy: 0.9571
E

Epoch: [ 0] [  449/  468] time: 85.0959, train_loss: 0.06594405, train_accuracy: 0.9844, test_Accuracy: 0.9592
Epoch: [ 0] [  450/  468] time: 85.3126, train_loss: 0.21401742, train_accuracy: 0.9375, test_Accuracy: 0.9573
Epoch: [ 0] [  451/  468] time: 85.5408, train_loss: 0.15255608, train_accuracy: 0.9609, test_Accuracy: 0.9564
Epoch: [ 0] [  452/  468] time: 85.7540, train_loss: 0.09830007, train_accuracy: 0.9609, test_Accuracy: 0.9573
Epoch: [ 0] [  453/  468] time: 85.9640, train_loss: 0.11358110, train_accuracy: 0.9766, test_Accuracy: 0.9594
Epoch: [ 0] [  454/  468] time: 86.1426, train_loss: 0.09549601, train_accuracy: 0.9688, test_Accuracy: 0.9603
Epoch: [ 0] [  455/  468] time: 86.3053, train_loss: 0.14264117, train_accuracy: 0.9531, test_Accuracy: 0.9617
Epoch: [ 0] [  456/  468] time: 86.4738, train_loss: 0.12987432, train_accuracy: 0.9688, test_Accuracy: 0.9633
Epoch: [ 0] [  457/  468] time: 86.6460, train_loss: 0.14170513, train_accuracy: 0.9609, test_Accuracy: 0.9635
E